# What are Decorators

### Notes/Interactively Running Examples From: http://thecodeship.com/patterns/guide-to-python-function-decorators/
Decorators are nothing more than syntactic sugar, but they provide a powerful functionality. They can dynamically alter the functionality of a function, method, or class without using subclasses or sub-functions.

Before we can discuss what decorators are, it might be useful to show what they accomplish using good old python.

Decorators are special wrappers which modify the behavior of a piece of code before, and after the target code's execution, but no modification is needed to the original function itself. Thus, a function is 'decorated'.

# Some Background On Functions

In python, functions themseles are 'first-class' citizens, they can be passed around like variables.

## Setting a Function to A Variable

In [1]:
def greet(name):
    """
    Takes a string, 'name' and returns a greeting string using that name.
    """
    return "hello {}".format(str(name))
# Look, we're assigning greet to a function!
greet_someone = greet
print(greet_someone("John"))
print(greet("Mike"))

hello John
hello Mike


## Defining Functions Inside of Functions

In [2]:
def greet_2(name):
    def get_message():
        return "Hello"
    result = get_message() + " " + name
    return result

print(greet_2("Kathy"))

Hello Kathy


## Passing a Function as a Parameter to Other Functions

In [3]:
def call_func(func, name):
    return func(name)

print(call_func(greet,"Jill"))

hello Jill


## Return a Function

In [4]:
def compose_greet_func():
    def get_message():
        return "Hello there!"
    return get_message

greet = compose_greet_func()
print(greet())

Hello there!


## A Few Notes on Scope: Decorators/Closures
Decorators are one expression of a __design pattern__ known as a __closure__. A closure is a design pattern where functions are 'aware' of the local enclosing environment when they are created.  

In [5]:
def compose_greet_func(name):
    def get_message():
        return "Hello there, {}!".format(name)
    return get_message

greet = compose_greet_func("John")
print(greet())

Hello there, John!


## Composing A Decorator
Note how get_message was wrapped inside of compose_greet_function. Note too that get_message (as seen directly above) has awareness of the scope in which it was created - i.e. it knows about 'name' without having to be explicitly told. We created a closure around get_message. 

Function decorators wrap exisitng functions. Putting the above ideas together, we can build a decorator. For example, consider a function that wraps the output of a string with paragraph tag "&lt;p&gt; text &lt;/p&gt;".

In [6]:
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)

def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper
text_gen = p_decorate(get_text)
print(text_gen("John"))

<p>lorem ipsum, John dolor sit amet</p>


See how we have 'decorated' get_text by writing a wrapper? We accomplished this by creating a wrapper inside p_decorate which called the func argument passed in, and inserted the output into the string.

A function that takes another function as an argument, generates a new function while augmenting the work of the original. The function p_decorate is LITERALLY a decorator pattern.

We can make get_text be decorated by p_decorate itself by assigning get_text to the result of p_decorate itself.}

In [7]:
# in a Jupyter Notebook Kernel, this can lead to recursively decorated results. Be Careful!
# This syntax is a little confusing - since p_decorate takes 'func' which is called as a function
# yet get_text expects a string-y argument.
get_text = p_decorate(get_text) 
print(get_text("John"))

<p>lorem ipsum, John dolor sit amet</p>


Note that get_text takes an argument 'name' but the decorator itself does not take any name argument. It is passed in through the wrapper, func_wrapper.

While we can still use this formalism to create decorators, python has added some syntactic sugar which makes the process a bit easier, so that we don't need the <pre> get_text = p_decorator(get_text) </pre> syntax. Instead we can create a short-cut for that, with the @ modifier.

In [8]:
import functools
def p_decorate(func):
    def func_wrapper(name):
        """ 
        wrapper for the function to be modified
        """
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

# same as later doing: get_text = p_decorator(get_text)
@p_decorate
def get_text(name):
    return "lorem ipsum {0} dolor sit amet".format(name)

print(get_text("John"))


<p>lorem ipsum John dolor sit amet</p>


The order of setting decorators matters, FYI.

## Decorating Methods
Methods in python are just a special class of functions which must have at least one argument - a pointer to the class that they belong to - i.e. 'self'. Decorators can be built for methods, taking 'self' into account.

In [9]:
def p_decorate(func):
    def func_wrapper(self): # note that here we give the argument self
        return "<p>{0}</p>".format(func(self))
    return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"
    
    # or method = p_decorate(method)
    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family
    
my_person = Person()
print(my_person.get_fullname())    

<p>John Doe</p>


What if we wanted to make a decorator that could handle functions _or_ methods? It can be done using &#42;args and &#42;&#42;kwargs. Previously, a decorator function needed to know something about the arguments to be supplied to the function it would decorate. However, &#42;args and &#42;&#42;kwargs allow us to essentially handle any number of arguments (and they are implicitly the arguments passed to every function anyway). &#42;args is simply a tuple of unnamed arguments in the order that they were received.  &#42;kwargs stands for 'key-word' arguments and is a dict of key-value pairs. The key is cast as a string internally.

In [10]:
def show_me(*args,**kwargs):
    print(type(args),args)
    print(type(kwargs),kwargs)
    for arg in args:
        print("arg:",arg)
    for kw,arg in kwargs.items():
        print("kw:",kw,"arg:",arg)
    return
show_me(1,2,3,4)
show_me(one=1, two=2, three=3, four=4)

<class 'tuple'> (1, 2, 3, 4)
<class 'dict'> {}
arg: 1
arg: 2
arg: 3
arg: 4
<class 'tuple'> ()
<class 'dict'> {'one': 1, 'four': 4, 'three': 3, 'two': 2}
kw: one arg: 1
kw: four arg: 4
kw: three arg: 3
kw: two arg: 2


In [11]:
def p_decorate(func):
    def func_wrapper(*args, **kwargs):
        return "<p>{0}</p>".format(func(*args, **kwargs))
    return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"
    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family
    
my_person = Person()

print(my_person.get_fullname())

<p>John Doe</p>


## Modifying Decorators With Arguments
Just like any other function, decorators can be given arguments as well. This might be useful in the case where we wanted to simply abstract doing a similar thing under different conditions. For example, in this case, we have been playing with using decorators to add HTML tags to strings. Why should we have a 'p_decorator' to apply &gt;p^&lt; tags when we could generalize this decorator to add any tag we might want?

In [17]:
def tag_decorate(tag_name): # supply the decorator argument one level up
    def tags_decorator(func): # take the outer function as an argument here
        def func_wrapper(*args, **kwargs): # any arguments the inner function may need here
            return "<{0}>{1}</{0}>".format(tag_name, func(*args,**kwargs))
        return func_wrapper
    return tags_decorator

# decorate with 'p'
@tag_decorate("p")
def get_text(name):
    return "Hello "+name

print(get_text("John"))

@tag_decorate("strong")
def get_text(name):
    return "Hello "+name

print(get_text("Mike"))

@tag_decorate("poop")
def twenty_seven():
    return 27

print(twenty_seven())

<p>Hello John</p>
<strong>Hello Mike</strong>
<poop>27</poop>


## Debugging
Decorators wrap the function, so debugging a decorated function can be problematic. Functools.wraps can elp with this for updating the attributes of the original function.

In [6]:
from functools import wraps

def tags(tag_name):
    def tags_decorator(func):
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name,func(name))
        return func_wrapper
    return tags_decorator

def tags_protected(tag_name):
    def tags_decorator(func):
        @wraps(func)
        # changes the function, and returns the decorator after 
        # passing in the funciton's argument.
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name,func(name))
        return func_wrapper
    return tags_decorator


@tags("p")
def get_text(name):
    """returns some text"""
    return "Hello "+name

@tags_protected("p")
def get_text_protected(name):
    """returns some text"""
    return "Hello "+name

# manual decoration
def fun_ction(name):
    """returns some text"""
    return "Hello, "+name
print(80*"*")
print( "UNPROTECTED" )
print( get_text.__name__ )
print( get_text.__doc__ )
print( get_text.__module__ )
print(80*"*")
print( "PROTECTED" )
print( get_text_protected.__name__ )
print( get_text_protected.__doc__ )
print( get_text_protected.__module__ )
print(80*"*")
# manually decorate:
decorator_function = tags_protected("poo")
decorator_function(fun_ction)
print(caller.__name__)
print(caller.__doc__)
print(caller.__module__)
print(caller("Mike"))

********************************************************************************
UNPROTECTED
func_wrapper
None
__main__
********************************************************************************
PROTECTED
get_text_protected
returns some text
__main__
********************************************************************************
fun_ction
returns some text
__main__
<poo>Hello, Mike</poo>


# Next Steps
Some useful links:

- <a href = "https://wiki.python.org/moin/PythonDecoratorLibrary"> Python Decorator Library </a>
- <a href = "https://wiki.python.org/moin/PythonDecorators#What_is_a_Decorator"> Definition of a Decorator </a>
- <a href = "http://www.artima.com/weblogs/viewpost.jsp?thread=240808"> Introduction to Decorators </a>
- <a href = "http://www.artima.com/weblogs/viewpost.jsp?thread=240845"> Decorator Arguements </a>
- <a href = "http://www.artima.com/weblogs/viewpost.jsp?thread=241209"> Decorator Build System </a>